## SET UP
***

In [ ]:
#import relevant libraries
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import random
from random import sample
random.seed(1337)
import matplotlib.animation as animation
from matplotlib.colors import ListedColormap
from csv import writer

In [ ]:
#we will use the manhattan distance for our main metric in the lattice, using periodic boundary conditions
def d(x,y):
    vert_diff = min((x[0]-y[0])%N, (y[0]-x[0])%N)
    horiz_diff = min((x[1]-y[1])%N, (y[1]-x[1])%N)
    return vert_diff + horiz_diff

#finds the Von Neumann neighbourhood of radius r of a given coordinate, using periodic boundary conditions
def vn_nhd(x, r):
    neighbours = []
    for j in range(-(r),r+1):
        neighbours.append([x[0] % N,(x[1]+j) % N])
    for i in range(1,r+1):
        for j in range(-(r-i),r+1-i):
            neighbours.append([(x[0]-i) % N,(x[1]+j) % N])
            neighbours.append([(x[0]+i) % N,(x[1]+j) % N])
    return(neighbours)

#returns a random distinct neighbour to the subject in their von neumann neighbourhood
def random_neighbour(x,vn_nhd_size):
    candidates = vn_nhd(x,vn_nhd_size)
    neighbour = x
    while neighbour[0] == x[0] and neighbour[1] == x[1]:
        neighbour = candidates[random.randint(0,len(candidates)-1)]
    return neighbour

#testing
N = 40
x = [0,0]
print(x)
print(d(x, [4,17]))
print(len(vn_nhd(x,8)))

In [ ]:
#used to evaluate the expected number of people a person knows the reputation of
def expected_friends(end,theta,k):
    value = 0
    for i in range(1,end+1):
        value += 4*i/(1 + np.exp((i-theta)/k))
    return value


def prob_knows(x,y):
    return 1 / (1 + np.exp(d(x,y)/4-2))

#testing
print(expected_friends(1000,8,2))
print(d([2,0],[2,8]))
print(prob_knows([2,0],[2,17]))
print(prob_knows([0,0],[0,0]))

In [ ]:
N = 100
prob_values = np.linspace(0,20,40)
probs = [prob_knows([2,0],[2,prob_values[i]]) for i in range(len(prob_values))]

fig, ax = plt.subplots()
plt.title("Probability x knows y")
plt.ylabel("Probability")
plt.xlabel("d(x,y)")
ax.plot(prob_values, [0 for i in range(40)], linewidth=2.0, label = "Cooperators", color = "black", linestyle = "dashed")
ax.plot(prob_values, probs, linewidth=2.0, label = "Cooperators", color = "olive")
plt.show()

In [ ]:
#custom colour map for use in visual representations of strategies
cmap = ListedColormap(["#BC2023", "#094A25"], N=2)

## Round elements

In [ ]:
def public_goods(players,t):
    communal_pot = 0
    #calculate how much money in communal funds 
    for player in players:
        if strategy[t][player[0]][player[1]] == 1: #cooperators donate to communal pot
            communal_pot += min(payoff[t][player[0]][player[1]], donation_PG) #players can't donate more than they have
            payoff[t][player[0]][player[1]] += - min(payoff[t][player[0]][player[1]], donation_PG)
            reputation[t][player[0]][player[1]] = 1
            
        elif strategy[t][player[0]][player[1]] == 0: #defectors lose reputation
            reputation[t][player[0]][player[1]] = 0
    
    for player in players: #everyone gets the money from the communal pot split
        payoff[t][player[0]][player[1]] += communal_pot * r / len(players)
        payoff[t][player[0]][player[1]] = round(payoff[t][player[0]][player[1]],2) #round to 2dp

In [ ]:
def indirect_reciprocity(donor,recipient,t):
    if reputation[t][recipient[0]][recipient[1]] == 1:
        payoff[t][recipient[0]][recipient[1]] += benefit_IR
        payoff[t][donor[0]][donor[1]] += cost_IR
        cum_donations[t] += 1
        reputation[t][donor[0]][donor[1]] = 1
        
def indirect_reciprocity_spatial(donor,recipient,t):
    rand_no = np.random.uniform(0,1)
    if rand_no < prob_knows(donor,recipient): #donor knows recipient's reputation
        indirect_reciprocity(donor,recipient,t)
        cum_rep_known[t] += 1   #always help a good rep and get good rep, refuse to help bad rep and rep unchanged
    
    else: #always donate if you don't know reputation
        payoff[t][recipient[0]][recipient[1]] += benefit_IR
        payoff[t][donor[0]][donor[1]] += cost_IR
        cum_donations[t] += 1
        reputation[t][donor[0]][donor[1]] = 1


In [ ]:
#compare the payoff of a subject x with a neighbour y
#change x's strategy to the y's if the payoff is strictly higher
def update_rule(x,y,t):
    #introduce variables for readability
    x_payoff = payoff[t][x[0]][x[1]] 
    y_payoff = payoff[t][y[0]][y[1]]
    


    
    #update rule applied if y has strictly greater payoff and different strategy
    if y_payoff > x_payoff and strategy[t][x[0]][x[1]] != strategy[t][y[0]][y[1]]: 
        strategy[t][x[0]][x[1]] = strategy[t][y[0]][y[1]] #x swaps strategy to y's
        
        if strategy[t][x[0]][x[1]] == 1: #if x swapped to 1
            no_cooperators[t] += 1
            cum_dc_swaps[t] += 1
        
        elif strategy[t][x[0]][x[1]] == 0: #if x swapped to 0
            no_cooperators[t] += -1
            cum_cd_swaps[t] += 1

        


## Public Goods Only

In [ ]:
#PARAMETERS FOR "HOW LONG DOES IT TAKE FOR COOPERATORS TO BE MADE EXTINCT IN PG"

N = 40 #for use in the NxN lattice, total population size = N^2
n = np.power(N,2) #population size
p = 10 #each player starts with £p
benefit_IR = 0 #benefit received by recipient
cost_IR = 0 #cost to donor
donation_PG = 1.25 #amount a cooperator pays
r = 2 #cooperators give donation_PG then the total amount is multiplied by r where 1 < r < n = N^2 and divided by everyone
IR_range = 1
no_rounds = 40000
no_trials = 10
spatial = False

import time
start_time = time.time()



for trial in range(no_trials):
    #the three main properties that any individual has
    #e.g. reputation[t] = the reputation matrix after the t-th round
    #reputation[0] = the initial reputation matrix
    reputation = [[] for i in range(no_rounds+1)]  
    strategy = [[] for i in range(no_rounds+1)] 
    payoff = [[] for i in range(no_rounds+1)] 

    #metrics used for evalutation
    no_cooperators = np.zeros(no_rounds+1) 
    av_defector_payoff = np.zeros(no_rounds+1) 
    av_cooperator_payoff = np.zeros(no_rounds+1) 
    cum_donations = np.zeros(no_rounds+1)  
    cum_cd_swaps = np.zeros(no_rounds+1)  
    cum_dc_swaps = np.zeros(no_rounds+1) 
    cum_rep_known = np.zeros(no_rounds+1)

    #initial values
    reputation[0] = np.ones([N,N]) #everyone starts with a reputation of 1
    strategy[0] = np.random.randint(2, size=(N, N), ) #uniformly random distributed strategies
    payoff[0] = np.array([[p for i in range(N)] for j in range(N)], dtype=float) #everyone starts with the same amount of money
    no_cooperators[0] = np.sum(strategy[0])
    av_defector_payoff[0] = p
    av_cooperator_payoff[0] = p

    print(no_cooperators[0])
    keep_running = True
    while (keep_running): #Only keeps running if non-0 number of cooperators

        #repeat round for every time step
        for t in range(1,no_rounds+1):
            #copy across all properties for new time step
            reputation[t] = reputation[t-1].copy()
            strategy[t] = strategy[t-1].copy()
            payoff[t] = payoff[t-1].copy()
            no_cooperators[t] = no_cooperators[t-1].copy()
            cum_cd_swaps[t] = cum_cd_swaps[t-1]
            cum_dc_swaps[t] = cum_dc_swaps[t-1]
            cum_donations[t] = cum_donations[t-1]
            cum_rep_known[t] = cum_rep_known[t-1]
            
            if t % 2000 ==0:
                print(t, no_cooperators[t],"cooperators")

            #public goods 5x with the focus of each game being a member of x's 1-vn-nhd
            x = np.random.randint(N,size=[2])
            for focus in vn_nhd(x, 1):
                public_goods(vn_nhd(focus,1),t)

            #indirect reciprocity 5x with the recipient of each game being a member of x's 1-vn-nhd
            for focus in vn_nhd(x, 1):
                if spatial == True:
                    indirect_reciprocity_spatial(random_neighbour(focus,IR_range), focus, t) #focus = recipient, random player in lattice = donor
                else:
                    indirect_reciprocity(random_neighbour(focus,IR_range), focus, t)

            #x decides whether to emulate neighbour's strategy or not
            update_rule(x,random_neighbour(x,1),t)

            #work out average payoff of defector and cooperators
            for i in range(N):
                for j in range(N):
                    if strategy[t][i][j] == 1:
                        av_cooperator_payoff[t] += payoff[t][i][j]/no_cooperators[t]
                    elif strategy[t][i][j] == 0:
                        av_defector_payoff[t] += payoff[t][i][j]/(n-no_cooperators[t])
            
            if (no_cooperators[t])*(n-no_cooperators[t]) == 0:
                keep_running = False
                break
            
        with open('PG_defectors_dominate.csv', 'a') as f:
                    writer_ob = writer(f)
                    if t == n:
                        writer_ob.writerow([Inf])
                    else:
                        writer_ob.writerow([t])
                    f.close()
        
        print("\ntrial", trial, "ends at", t,"\n")


print(time.time() - start_time)

Note that the process for all of these sections running models is essentially the same - the code blocks are seperated out to allow for different metrics to be calculated and different files to be produced. No. trials must be set manually.

## Alternating Model

In [ ]:
#PARAMETERS FOR BASE AM
N = 40 #for use in the NxN lattice, total population size = N^2
n = np.power(N,2) #population size
p = 10 #each player starts with £p
benefit_IR = 2 #benefit received by recipient
cost_IR = 1.25 #cost to donor
donation_PG = 1.25 #amount a cooperator pays
r = 2 #cooperators give donation_PG then the total amount is multiplied by r where 1 < r < n = N^2 and divided by everyone
IR_range = 8
no_rounds = 40000
no_trials = 100
spatial = False

import time
start_time = time.time()




for trial in range(no_trials):
    #the three main properties that any individual has
    reputation = [[] for i in range(no_rounds+1)]  
    strategy = [[] for i in range(no_rounds+1)] 
    payoff = [[] for i in range(no_rounds+1)] 

    #metrics used for evalutation
    no_cooperators = np.zeros(no_rounds+1) 
    av_defector_payoff = np.zeros(no_rounds+1) 
    av_cooperator_payoff = np.zeros(no_rounds+1) 
    cum_donations = np.zeros(no_rounds+1)  
    cum_cd_swaps = np.zeros(no_rounds+1)  
    cum_dc_swaps = np.zeros(no_rounds+1) 
    cum_rep_known = np.zeros(no_rounds+1)

    #initial values
    reputation[0] = np.ones([N,N]) #everyone starts with a reputation of 1
    strategy[0] = np.random.randint(2, size=(N, N), ) #uniformly random distributed strategies
    payoff[0] = np.array([[p for i in range(N)] for j in range(N)], dtype=float) #everyone starts with the same amount of money
    no_cooperators[0] = np.sum(strategy[0])
    av_defector_payoff[0] = p
    av_cooperator_payoff[0] = p

    print(0, no_cooperators[0],"cooperators")
    #repeat round for every time step
    for t in range(1,no_rounds+1):
        #copy across all properties for new time step
        reputation[t] = reputation[t-1].copy()
        strategy[t] = strategy[t-1].copy()
        payoff[t] = payoff[t-1].copy()
        no_cooperators[t] = no_cooperators[t-1].copy()
        cum_cd_swaps[t] = cum_cd_swaps[t-1]
        cum_dc_swaps[t] = cum_dc_swaps[t-1]
        cum_donations[t] = cum_donations[t-1]
        cum_rep_known[t] = cum_rep_known[t-1]
            
        if t % 2000 ==0:
            print(t, no_cooperators[t],"cooperators")

        #public goods 5x with the focus of each game being a member of x's 1-vn-nhd
        x = np.random.randint(N,size=[2])
        for focus in vn_nhd(x, 1):
            public_goods(vn_nhd(focus,1),t)

        #indirect reciprocity 5x with the recipient of each game being a member of x's 1-vn-nhd
        for focus in vn_nhd(x, 1):
            if spatial == True:
                indirect_reciprocity_spatial(random_neighbour(focus,IR_range), focus, t) #focus = recipient, random player in lattice = donor
            else:
                indirect_reciprocity(random_neighbour(focus,IR_range), focus, t)

        #x decides whether to emulate neighbour's strategy or not
        update_rule(x,random_neighbour(x,1),t)

        #work out average payoff of defector and cooperators
        for i in range(N):
            for j in range(N):
                if strategy[t][i][j] == 1:
                    av_cooperator_payoff[t] += payoff[t][i][j]/no_cooperators[t]
                elif strategy[t][i][j] == 0:
                    av_defector_payoff[t] += payoff[t][i][j]/(n-no_cooperators[t])
            
    
    
    with open('IR_base_final_coop.csv', 'a') as f:
        writer_ob = writer(f)
        writer_ob.writerow([no_cooperators[no_rounds]])
        f.close()
        
    with open('IR_base_cooperators.csv', 'a') as f:
        writer_ob = writer(f)
        writer_ob.writerow([no_cooperators[i] for i in range(0, no_rounds + 100,100)])
        f.close()
        
    with open('IR_base_coop_payoff.csv', 'a') as f:
        writer_ob = writer(f)
        writer_ob.writerow([av_cooperator_payoff[i] for i in range(0, no_rounds + 100,100)])
        f.close()
    
    with open('IR_base_defec_payoff.csv', 'a') as f:
        writer_ob = writer(f)
        writer_ob.writerow([av_defector_payoff[i] for i in range(0, no_rounds + 100,100)])
        f.close()
    
    with open('IR_base_dc_swaps.csv', 'a') as f:
        writer_ob = writer(f)
        writer_ob.writerow([cum_dc_swaps[i] for i in range(0, no_rounds + 100,100)])
        f.close()
        
    with open('IR_base_cd_swaps.csv', 'a') as f:
        writer_ob = writer(f)
        writer_ob.writerow([cum_cd_swaps[i] for i in range(0, no_rounds + 100,100)])
        f.close()
        
    with open('IR_base_final_swaps.csv', 'a') as f:
        writer_ob = writer(f)
        writer_ob.writerow([cum_cd_swaps[no_rounds], cum_dc_swaps[no_rounds]])
        f.close()
        
    with open('IR_base_cum_don.csv', 'a') as f:
        writer_ob = writer(f)
        writer_ob.writerow([cum_donations[i] for i in range(0, no_rounds + 100,100)])
        f.close()
        
    with open('IR_base_final_don.csv', 'a') as f:
        writer_ob = writer(f)
        writer_ob.writerow([cum_donations[no_rounds]])
        f.close()
    
    with open('IR_base_cum_don.csv', 'a') as f:
        writer_ob = writer(f)
        writer_ob.writerow([cum_donations[i] for i in range(0, no_rounds + 100,100)])
        f.close()
     
    av_reputation = [1] + [no_cooperators[t-1]/n for t in range(1,no_rounds+1)]
    with open('IR_base_cum_don.csv', 'a') as f:
        writer_ob = writer(f)
        writer_ob.writerow([av_reputation[i] for i in range(0, no_rounds + 100,100)])
        f.close()

        
    print("\ntrial", trial, "finished with", no_cooperators[no_rounds],"\n")

print((time.time() - start_time)/3600, "hours")

8000 108.0 cooperators
10000 83.0 cooperators
12000 66.0 cooperators
14000 61.0 cooperators
16000 61.0 cooperators
18000 54.0 cooperators
20000 46.0 cooperators
22000 35.0 cooperators
24000 29.0 cooperators
26000 22.0 cooperators
28000 20.0 cooperators
30000 21.0 cooperators
32000 22.0 cooperators
34000 22.0 cooperators
36000 15.0 cooperators
38000 15.0 cooperators
40000 15.0 cooperators

trial 48 finished with 15.0 

0 798.0 cooperators
2000 464.0 cooperators
4000 252.0 cooperators
6000 139.0 cooperators
8000 74.0 cooperators
10000 48.0 cooperators
12000 50.0 cooperators
14000 38.0 cooperators
16000 42.0 cooperators
18000 37.0 cooperators
20000 39.0 cooperators
22000 43.0 cooperators
24000 46.0 cooperators
26000 53.0 cooperators
28000 58.0 cooperators
30000 67.0 cooperators
32000 74.0 cooperators
34000 70.0 cooperators
36000 76.0 cooperators
38000 93.0 cooperators
40000 92.0 cooperators

trial 49 finished with 92.0 

0 775.0 cooperators
2000 408.0 cooperators
4000 219.0 cooperators
60

8000 98.0 cooperators
10000 68.0 cooperators
12000 48.0 cooperators
14000 39.0 cooperators
16000 38.0 cooperators
18000 33.0 cooperators
20000 35.0 cooperators
22000 36.0 cooperators
24000 46.0 cooperators
26000 45.0 cooperators
28000 50.0 cooperators
30000 58.0 cooperators
32000 65.0 cooperators
34000 68.0 cooperators
36000 64.0 cooperators
38000 70.0 cooperators
40000 88.0 cooperators

trial 64 finished with 88.0 

0 816.0 cooperators
2000 479.0 cooperators
4000 293.0 cooperators
6000 166.0 cooperators
8000 109.0 cooperators
10000 84.0 cooperators
12000 62.0 cooperators
14000 62.0 cooperators
16000 58.0 cooperators
18000 62.0 cooperators
20000 79.0 cooperators
22000 88.0 cooperators
24000 98.0 cooperators
26000 98.0 cooperators
28000 106.0 cooperators
30000 119.0 cooperators
32000 126.0 cooperators
34000 134.0 cooperators
36000 135.0 cooperators
38000 138.0 cooperators
40000 154.0 cooperators

trial 65 finished with 154.0 

0 768.0 cooperators
2000 455.0 cooperators
4000 241.0 cooper

6000 191.0 cooperators
8000 113.0 cooperators
10000 77.0 cooperators
12000 49.0 cooperators
14000 42.0 cooperators
16000 34.0 cooperators
18000 35.0 cooperators
20000 30.0 cooperators
22000 37.0 cooperators
24000 36.0 cooperators
26000 32.0 cooperators
28000 33.0 cooperators
30000 37.0 cooperators
32000 37.0 cooperators
34000 37.0 cooperators
36000 37.0 cooperators
38000 36.0 cooperators
40000 43.0 cooperators

trial 80 finished with 43.0 

0 804.0 cooperators
2000 484.0 cooperators
4000 274.0 cooperators
6000 146.0 cooperators
8000 80.0 cooperators
10000 52.0 cooperators
12000 44.0 cooperators
14000 31.0 cooperators
16000 29.0 cooperators
18000 24.0 cooperators
20000 19.0 cooperators
22000 20.0 cooperators
24000 15.0 cooperators
26000 19.0 cooperators
28000 18.0 cooperators
30000 12.0 cooperators
32000 10.0 cooperators
34000 6.0 cooperators
36000 4.0 cooperators
38000 1.0 cooperators
40000 0.0 cooperators

trial 81 finished with 0.0 

0 786.0 cooperators
2000 477.0 cooperators
4000 26

8000 92.0 cooperators
10000 61.0 cooperators
12000 41.0 cooperators
14000 26.0 cooperators
16000 19.0 cooperators
18000 16.0 cooperators
20000 16.0 cooperators
22000 12.0 cooperators
24000 11.0 cooperators
26000 8.0 cooperators
28000 3.0 cooperators
30000 0.0 cooperators
32000 0.0 cooperators
34000 0.0 cooperators
36000 0.0 cooperators
38000 0.0 cooperators
40000 0.0 cooperators

trial 96 finished with 0.0 

0 797.0 cooperators
2000 475.0 cooperators
4000 272.0 cooperators
6000 165.0 cooperators
8000 101.0 cooperators
10000 65.0 cooperators
12000 50.0 cooperators
14000 41.0 cooperators
16000 29.0 cooperators
18000 31.0 cooperators
20000 32.0 cooperators
22000 41.0 cooperators
24000 45.0 cooperators
26000 45.0 cooperators
28000 45.0 cooperators
30000 49.0 cooperators
32000 56.0 cooperators
34000 49.0 cooperators
36000 59.0 cooperators
38000 76.0 cooperators
40000 83.0 cooperators

trial 97 finished with 83.0 

0 832.0 cooperators
2000 515.0 cooperators
4000 309.0 cooperators
6000 203.0 

In [ ]:
#PARAMETERS FOR AM RANGE
N = 40 #for use in the NxN lattice, total population size = N^2
n = np.power(N,2) #population size
p = 10 #each player starts with £p
benefit_IR = 15 #benefit received by recipient #will have to manually change this
cost_IR = 1.25 #cost to donor
donation_PG = 1.25 #amount a cooperator pays
r = 2 #cooperators give donation_PG then the total amount is multiplied by r where 1 < r < n = N^2 and divided by everyone
IR_range = 8
no_rounds = 40000
no_trials = 30
spatial = False

import time
start_time = time.time()

defectors_time = []
cooperators_time = []
av_cooperators = np.zeros(int(no_rounds/100+1))
final_don = []
final_cd = []
final_dc = []


for trial in range(no_trials):
    #the three main properties that any individual has
    reputation = [[] for i in range(no_rounds+1)]  
    strategy = [[] for i in range(no_rounds+1)] 
    payoff = [[] for i in range(no_rounds+1)] 

    #metrics used for evalutation
    no_cooperators = np.zeros(no_rounds+1) 
    av_defector_payoff = np.zeros(no_rounds+1) 
    av_cooperator_payoff = np.zeros(no_rounds+1) 
    cum_donations = np.zeros(no_rounds+1)  
    cum_cd_swaps = np.zeros(no_rounds+1)  
    cum_dc_swaps = np.zeros(no_rounds+1) 
    cum_rep_known = np.zeros(no_rounds+1)

    #initial values
    reputation[0] = np.ones([N,N]) #everyone starts with a reputation of 1
    strategy[0] = np.random.randint(2, size=(N, N), ) #uniformly random distributed strategies
    payoff[0] = np.array([[p for i in range(N)] for j in range(N)], dtype=float) #everyone starts with the same amount of money
    no_cooperators[0] = np.sum(strategy[0])
    av_defector_payoff[0] = p
    av_cooperator_payoff[0] = p

    print(0, no_cooperators[0],"cooperators")
    
    
    #repeat round for every time step
    for t in range(1,no_rounds+1):
        #copy across all properties for new time step
        reputation[t] = reputation[t-1].copy()
        strategy[t] = strategy[t-1].copy()
        payoff[t] = payoff[t-1].copy()
        no_cooperators[t] = no_cooperators[t-1].copy()
        cum_cd_swaps[t] = cum_cd_swaps[t-1]
        cum_dc_swaps[t] = cum_dc_swaps[t-1]
        cum_donations[t] = cum_donations[t-1]
        cum_rep_known[t] = cum_rep_known[t-1]
            
        if t % 2000 ==0:
            print(t, no_cooperators[t],"cooperators")

        #public goods 5x with the focus of each game being a member of x's 1-vn-nhd
        x = np.random.randint(N,size=[2])
        for focus in vn_nhd(x, 1):
            public_goods(vn_nhd(focus,1),t)

        #indirect reciprocity 5x with the recipient of each game being a member of x's 1-vn-nhd
        for focus in vn_nhd(x, 1):
            if spatial == True:
                indirect_reciprocity_spatial(random_neighbour(focus,IR_range), focus, t) #focus = recipient, random player in lattice = donor
            else:
                indirect_reciprocity(random_neighbour(focus,IR_range), focus, t)

        #x decides whether to emulate neighbour's strategy or not
        update_rule(x,random_neighbour(x,1),t)
            
    
    for i in range(0, no_rounds + 100,100):
        av_cooperators[int(i/100)] += no_cooperators[i]/no_trials

    
    extinct = np.where(no_cooperators==0)[0]
    if extinct.size > 1:
        defectors_time.append(extinct[0])
        
    extinct = np.where(no_cooperators==n)[0]
    if extinct.size > 1:
        cooperators_time.append(extinct[0])
    print(cooperators_time)
        
    final_don.append(cum_donations[no_rounds])
    final_cd.append(cum_cd_swaps[no_rounds])
    final_dc.append(cum_dc_swaps[no_rounds])
    


        
    print("\ntrial", trial, "finished with", no_cooperators[no_rounds],"\n")

print(cooperators_time)
with open('IR_range_dom.csv', 'a') as f:
    writer_ob = writer(f)
    writer_ob.writerow([benefit_IR, defectors_time, cooperators_time])
    f.close()
    
with open('IR_range_cooperators.csv', 'a') as f:
    writer_ob = writer(f)
    writer_ob.writerow([benefit_IR, av_cooperators])
    f.close()
    
with open('IR_range_don.csv', 'a') as f:
    writer_ob = writer(f)
    writer_ob.writerow([benefit_IR, np.mean(final_don)])
    f.close()
        
with open('IR_range_swaps.csv', 'a') as f:
    writer_ob = writer(f)
    writer_ob.writerow([benefit_IR, np.mean(final_dc), np.mean(final_cd)])
    f.close()

print((time.time() - start_time)/3600, "hours")

34000 1600.0 cooperators
36000 1600.0 cooperators
38000 1600.0 cooperators
40000 1600.0 cooperators
[11217, 19705, 12588, 14354, 17008, 13923, 13491, 15416, 15676, 13946, 16310, 14267, 14049, 16057]

trial 13 finished with 1600.0 

0 812.0 cooperators
2000 1212.0 cooperators
4000 1419.0 cooperators
6000 1531.0 cooperators
8000 1585.0 cooperators
10000 1598.0 cooperators
12000 1600.0 cooperators
14000 1600.0 cooperators
16000 1600.0 cooperators
18000 1600.0 cooperators
20000 1600.0 cooperators
22000 1600.0 cooperators
24000 1600.0 cooperators
26000 1600.0 cooperators
28000 1600.0 cooperators
30000 1600.0 cooperators
32000 1600.0 cooperators
34000 1600.0 cooperators
36000 1600.0 cooperators
38000 1600.0 cooperators
40000 1600.0 cooperators
[11217, 19705, 12588, 14354, 17008, 13923, 13491, 15416, 15676, 13946, 16310, 14267, 14049, 16057, 11230]

trial 14 finished with 1600.0 

0 818.0 cooperators
2000 1237.0 cooperators
4000 1454.0 cooperators
6000 1547.0 cooperators
8000 1578.0 cooperato

32000 1600.0 cooperators
34000 1600.0 cooperators
36000 1600.0 cooperators
38000 1600.0 cooperators
40000 1600.0 cooperators
[11217, 19705, 12588, 14354, 17008, 13923, 13491, 15416, 15676, 13946, 16310, 14267, 14049, 16057, 11230, 15018, 12360, 12351, 11390, 12423, 17364, 12484, 17590, 16329, 12921, 18671]

trial 25 finished with 1600.0 

0 808.0 cooperators
2000 1223.0 cooperators
4000 1461.0 cooperators
6000 1547.0 cooperators
8000 1581.0 cooperators
10000 1595.0 cooperators
12000 1598.0 cooperators
14000 1599.0 cooperators
16000 1600.0 cooperators
18000 1600.0 cooperators
20000 1600.0 cooperators
22000 1600.0 cooperators
24000 1600.0 cooperators
26000 1600.0 cooperators
28000 1600.0 cooperators
30000 1600.0 cooperators
32000 1600.0 cooperators
34000 1600.0 cooperators
36000 1600.0 cooperators
38000 1600.0 cooperators
40000 1600.0 cooperators
[11217, 19705, 12588, 14354, 17008, 13923, 13491, 15416, 15676, 13946, 16310, 14267, 14049, 16057, 11230, 15018, 12360, 12351, 11390, 12423, 17

## Spatial Alternating Model

In [ ]:
#PARAMETERS FOR SPATIAL
N = 40 #for use in the NxN lattice, total population size = N^2
n = np.power(N,2) #population size
p = 10 #each player starts with £p
benefit_IR = 2 #benefit received by recipient
cost_IR = 1.25 #cost to donor
donation_PG = 1.25 #amount a cooperator pays
r = 2 #cooperators give donation_PG then the total amount is multiplied by r and divided by everyone
IR_range = 8
no_rounds = 40000
no_trials = 63
spatial = True

import time
start_time = time.time()




for trial in range(no_trials):
    #the three main properties that any individual has
    reputation = [[] for i in range(no_rounds+1)]  
    strategy = [[] for i in range(no_rounds+1)] 
    payoff = [[] for i in range(no_rounds+1)] 

    #metrics used for evalutation
    no_cooperators = np.zeros(no_rounds+1) 
    av_defector_payoff = np.zeros(no_rounds+1) 
    av_cooperator_payoff = np.zeros(no_rounds+1) 
    cum_donations = np.zeros(no_rounds+1)  
    cum_cd_swaps = np.zeros(no_rounds+1)  
    cum_dc_swaps = np.zeros(no_rounds+1) 
    cum_rep_known = np.zeros(no_rounds+1)

    #initial values
    reputation[0] = np.ones([N,N]) #everyone starts with a reputation of 1
    strategy[0] = np.random.randint(2, size=(N, N), ) #uniformly random distributed strategies
    payoff[0] = np.array([[p for i in range(N)] for j in range(N)], dtype=float) 
    #everyone starts with the same amount of money
    no_cooperators[0] = np.sum(strategy[0])
    av_defector_payoff[0] = p
    av_cooperator_payoff[0] = p

    print(0, no_cooperators[0],"cooperators")
    #repeat round for every time step
    for t in range(1,no_rounds+1):
        #copy across all properties for new time step
        reputation[t] = reputation[t-1].copy()
        strategy[t] = strategy[t-1].copy()
        payoff[t] = payoff[t-1].copy()
        no_cooperators[t] = no_cooperators[t-1].copy()
        cum_cd_swaps[t] = cum_cd_swaps[t-1]
        cum_dc_swaps[t] = cum_dc_swaps[t-1]
        cum_donations[t] = cum_donations[t-1]
        cum_rep_known[t] = cum_rep_known[t-1]
            
        if t % 2000 ==0:
            print(t, cum_donations[t],"donations")

        #public goods 5x with the focus of each game being a member of x's 1-vn-nhd
        x = np.random.randint(N,size=[2])
        for focus in vn_nhd(x, 1):
            public_goods(vn_nhd(focus,1),t)

        #indirect reciprocity 5x with the recipient of each game being a member of x's 1-vn-nhd
        for focus in vn_nhd(x, 1):
            if spatial == True:
                indirect_reciprocity_spatial(random_neighbour(focus,IR_range), focus, t)
                #focus = recipient, random player in lattice = donor
            else:
                indirect_reciprocity(random_neighbour(focus,IR_range), focus, t)

        #x decides whether to emulate neighbour's strategy or not
        update_rule(x,random_neighbour(x,1),t)

        #work out average payoff of defector and cooperators
        for i in range(N):
            for j in range(N):
                if strategy[t][i][j] == 1:
                    av_cooperator_payoff[t] += payoff[t][i][j]/no_cooperators[t]
                elif strategy[t][i][j] == 0:
                    av_defector_payoff[t] += payoff[t][i][j]/(n-no_cooperators[t])
            

    with open('Spatial_base_final_coop.csv', 'a') as f:
        writer_ob = writer(f)
        writer_ob.writerow([no_cooperators[no_rounds]])
        f.close()
        
    with open('Spatial_base_cooperators.csv', 'a') as f:
        writer_ob = writer(f)
        writer_ob.writerow([no_cooperators[i] for i in range(0, no_rounds + 100,100)])
        f.close()
        
    with open('Spatial_base_coop_payoff.csv', 'a') as f:
        writer_ob = writer(f)
        writer_ob.writerow([av_cooperator_payoff[i] for i in range(0, no_rounds + 100,100)])
        f.close()
    
    with open('Spatial_base_defec_payoff.csv', 'a') as f:
        writer_ob = writer(f)
        writer_ob.writerow([av_defector_payoff[i] for i in range(0, no_rounds + 100,100)])
        f.close()
    
    with open('Spatial_base_dc_swaps.csv', 'a') as f:
        writer_ob = writer(f)
        writer_ob.writerow([cum_dc_swaps[i] for i in range(0, no_rounds + 100,100)])
        f.close()
        
    with open('Spatial_base_cd_swaps.csv', 'a') as f:
        writer_ob = writer(f)
        writer_ob.writerow([cum_cd_swaps[i] for i in range(0, no_rounds + 100,100)])
        f.close()
        
    with open('Spatial_base_final_swaps.csv', 'a') as f:
        writer_ob = writer(f)
        writer_ob.writerow([cum_cd_swaps[no_rounds], cum_dc_swaps[no_rounds]])
        f.close()
    
  
    with open('Spatial_base_final_don.csv', 'a') as f:
        writer_ob = writer(f)
        writer_ob.writerow([cum_donations[no_rounds]])
        f.close()
    
    with open('Spatial_base_cum_don.csv', 'a') as f:
        writer_ob = writer(f)
        writer_ob.writerow([cum_donations[i] for i in range(0, no_rounds + 100,100)])
        f.close()
     
    av_reputation = [1] + [no_cooperators[t-1]/n for t in range(1,no_rounds+1)]
    with open('Spatial_base_cum_don.csv', 'a') as f:
        writer_ob = writer(f)
        writer_ob.writerow([av_reputation[i] for i in range(0, no_rounds + 100,100)])
        f.close()
        
    with open('Spatial_base_cum_rep_known.csv', 'a') as f:
        writer_ob = writer(f)
        writer_ob.writerow([cum_rep_known[i] for i in range(0, no_rounds + 100,100)])
        f.close()

     
    print("\ntrial", trial, "finished with", no_cooperators[no_rounds],"\n")

print((time.time() - start_time)/3600, "hours")

30000 48385.0 donations
32000 51163.0 donations
34000 54003.0 donations
36000 56835.0 donations
38000 59695.0 donations
40000 62477.0 donations

trial 15 finished with 4.0 

0 798.0 cooperators
2000 5565.0 donations
4000 9866.0 donations
6000 13586.0 donations
8000 16740.0 donations
10000 19812.0 donations
12000 22707.0 donations
14000 25572.0 donations
16000 28343.0 donations
18000 31155.0 donations
20000 34058.0 donations
22000 36856.0 donations
24000 39635.0 donations
26000 42510.0 donations
28000 45337.0 donations
30000 48181.0 donations
32000 50902.0 donations
34000 53773.0 donations
36000 56565.0 donations
38000 59289.0 donations
40000 62106.0 donations

trial 16 finished with 0.0 

0 807.0 cooperators
2000 5672.0 donations
4000 10043.0 donations
6000 13613.0 donations
8000 16927.0 donations
10000 19915.0 donations
12000 22890.0 donations
14000 25765.0 donations
16000 28694.0 donations
18000 31612.0 donations
20000 34577.0 donations
22000 37498.0 donations
24000 40452.0 donations

14000 25896.0 donations
16000 28819.0 donations
18000 31840.0 donations
20000 34805.0 donations
22000 37714.0 donations
24000 40637.0 donations
26000 43606.0 donations
28000 46518.0 donations
30000 49420.0 donations
32000 52275.0 donations
34000 55123.0 donations
36000 58078.0 donations
38000 60982.0 donations
40000 63937.0 donations

trial 31 finished with 35.0 

0 798.0 cooperators
2000 5670.0 donations
4000 10025.0 donations
6000 13730.0 donations
8000 17059.0 donations
10000 20035.0 donations
12000 22994.0 donations
14000 25926.0 donations
16000 28899.0 donations
18000 31837.0 donations
20000 34834.0 donations
22000 37876.0 donations
24000 41064.0 donations
26000 44087.0 donations
28000 47287.0 donations
30000 50435.0 donations
32000 53647.0 donations
34000 56888.0 donations
36000 60050.0 donations
38000 63265.0 donations
40000 66365.0 donations

trial 32 finished with 94.0 

0 780.0 cooperators
2000 5450.0 donations
4000 9751.0 donations
6000 13339.0 donations
8000 16624.0 donatio

0 770.0 cooperators
2000 5539.0 donations
4000 9872.0 donations
6000 13434.0 donations
8000 16644.0 donations
10000 19795.0 donations
12000 22781.0 donations
14000 25739.0 donations
16000 28667.0 donations
18000 31544.0 donations
20000 34444.0 donations
22000 37252.0 donations
24000 40044.0 donations
26000 42914.0 donations
28000 45780.0 donations
30000 48652.0 donations
32000 51336.0 donations
34000 54198.0 donations
36000 57023.0 donations
38000 59806.0 donations
40000 62502.0 donations

trial 47 finished with 3.0 

0 773.0 cooperators
2000 5629.0 donations
4000 9890.0 donations
6000 13497.0 donations
8000 16650.0 donations
10000 19729.0 donations
12000 22800.0 donations
14000 25720.0 donations
16000 28538.0 donations
18000 31408.0 donations
20000 34241.0 donations
22000 37062.0 donations
24000 39887.0 donations
26000 42818.0 donations
28000 45738.0 donations
30000 48592.0 donations
32000 51430.0 donations
34000 54299.0 donations
36000 57188.0 donations
38000 60192.0 donations
40000 

28000 45892.0 donations
30000 48753.0 donations
32000 51655.0 donations
34000 54617.0 donations
36000 57636.0 donations
38000 60559.0 donations
40000 63522.0 donations

trial 62 finished with 35.0 

8.887104448212517 hours


In [ ]:
#PARAMETERS FOR SPATIAL RANGE
N = 40 #for use in the NxN lattice, total population size = N^2
n = np.power(N,2) #population size
p = 10 #each player starts with £p
benefit_IR = 15 #benefit received by recipient #will have to manually change this
cost_IR = 1.25 #cost to donor
donation_PG = 1.25 #amount a cooperator pays
r = 2 #cooperators give donation_PG then the total amount is multiplied by r 
#where 1 < r < n = N^2 and divided by everyone
IR_range = 8
no_rounds = 40000
no_trials = 30
spatial = True

import time
start_time = time.time()

defectors_time = []
cooperators_time = []
av_cooperators = np.zeros(int(no_rounds/100+1))
final_don = []
final_cd = []
final_dc = []


for trial in range(no_trials):
    #the three main properties that any individual has
    reputation = [[] for i in range(no_rounds+1)]  
    strategy = [[] for i in range(no_rounds+1)] 
    payoff = [[] for i in range(no_rounds+1)] 

    #metrics used for evalutation
    no_cooperators = np.zeros(no_rounds+1) 
    av_defector_payoff = np.zeros(no_rounds+1) 
    av_cooperator_payoff = np.zeros(no_rounds+1) 
    cum_donations = np.zeros(no_rounds+1)  
    cum_cd_swaps = np.zeros(no_rounds+1)  
    cum_dc_swaps = np.zeros(no_rounds+1) 
    cum_rep_known = np.zeros(no_rounds+1)

    #initial values
    reputation[0] = np.ones([N,N]) #everyone starts with a reputation of 1
    strategy[0] = np.random.randint(2, size=(N, N), ) #uniformly random distributed strategies
    payoff[0] = np.array([[p for i in range(N)] for j in range(N)], dtype=float) #everyone starts with the same amount of money
    no_cooperators[0] = np.sum(strategy[0])
    av_defector_payoff[0] = p
    av_cooperator_payoff[0] = p

    print(0, no_cooperators[0],"cooperators")
    
    
    #repeat round for every time step
    for t in range(1,no_rounds+1):
        #copy across all properties for new time step
        reputation[t] = reputation[t-1].copy()
        strategy[t] = strategy[t-1].copy()
        payoff[t] = payoff[t-1].copy()
        no_cooperators[t] = no_cooperators[t-1].copy()
        cum_cd_swaps[t] = cum_cd_swaps[t-1]
        cum_dc_swaps[t] = cum_dc_swaps[t-1]
        cum_donations[t] = cum_donations[t-1]
        cum_rep_known[t] = cum_rep_known[t-1]
            
        if t % 2000 ==0:
            print(t, no_cooperators[t],"cooperators")

        #public goods 5x with the focus of each game being a member of x's 1-vn-nhd
        x = np.random.randint(N,size=[2])
        for focus in vn_nhd(x, 1):
            public_goods(vn_nhd(focus,1),t)

        #indirect reciprocity 5x with the recipient of each game being a member of x's 1-vn-nhd
        for focus in vn_nhd(x, 1):
            if spatial == True:
                indirect_reciprocity_spatial(random_neighbour(focus,IR_range), focus, t) 
                #focus = recipient, random player in lattice = donor
            else:
                indirect_reciprocity(random_neighbour(focus,IR_range), focus, t)

        #x decides whether to emulate neighbour's strategy or not
        update_rule(x,random_neighbour(x,1),t)
            
    
    for i in range(0, no_rounds + 100,100):
        av_cooperators[int(i/100)] += no_cooperators[i]/no_trials
    
    extinct = np.where(no_cooperators==0)[0]
    if extinct.size > 1:
        defectors_time.append(extinct[0])
        
    extinct = np.where(no_cooperators==n)[0]
    if extinct.size > 1:
        cooperators_time.append(extinct[0])
    print(cooperators_time)
        
    final_don.append(cum_donations[no_rounds])
    final_cd.append(cum_cd_swaps[no_rounds])
    final_dc.append(cum_dc_swaps[no_rounds])
    


        
    print("\ntrial", trial, "finished with", no_cooperators[no_rounds],"\n")

print(cooperators_time)
with open('Spatial_range_dom.csv', 'a') as f:
    writer_ob = writer(f)
    writer_ob.writerow([benefit_IR, defectors_time, cooperators_time])
    f.close()
    
with open('Spatial_range_cooperators.csv', 'a') as f:
    writer_ob = writer(f)
    writer_ob.writerow([benefit_IR, av_cooperators])
    f.close()
    
with open('Spatial_range_don.csv', 'a') as f:
    writer_ob = writer(f)
    writer_ob.writerow([benefit_IR, np.mean(final_don)])
    f.close()
        
with open('Spatial_range_swaps.csv', 'a') as f:
    writer_ob = writer(f)
    writer_ob.writerow([benefit_IR, np.mean(final_dc), np.mean(final_cd)])
    f.close()

print((time.time() - start_time)/3600, "hours")

## Creating Lattice Visualisations for Single Runs

In [ ]:
#GENERATING PIXEL

N = 40 #for use in the NxN lattice, total population size = N^2
n = np.power(N,2) #population size
p = 10 #each player starts with £p
benefit_IR = 0 #benefit received by recipient
cost_IR = 0 #cost to donor
donation_PG = 1.25 #amount a cooperator pays
r = 2 #cooperators give donation_PG then the total amount is multiplied by r where 1 < r < n = N^2 and divided by everyone
IR_range = 1
no_rounds = 40000
spatial = False

import time
start_time = time.time()


#the three main properties that any individual has
reputation = [[] for i in range(no_rounds+1)]  
strategy = [[] for i in range(no_rounds+1)] 
payoff = [[] for i in range(no_rounds+1)] 


#initial values
reputation[0] = np.ones([N,N]) #everyone starts with a reputation of 1
strategy[0] = np.random.randint(2, size=(N, N), ) 
#uniformly random distributed strategies
payoff[0] = np.array([[p for i in range(N)] for j in range(N)], dtype=float) 
#everyone starts with the same amount of money

no_cooperators = np.zeros(no_rounds+1) 
no_cooperators[0] = np.sum(strategy[0])



print(0, no_cooperators[0],"cooperators")


#repeat round for every time step
for t in range(1,no_rounds+1):
    #copy across all properties for new time step
    reputation[t] = reputation[t-1].copy()
    strategy[t] = strategy[t-1].copy()
    payoff[t] = payoff[t-1].copy()
    no_cooperators[t] = no_cooperators[t-1].copy()


    if t % 2000 ==0:
        print(t, no_cooperators[t],"cooperators")

    #public goods 5x with the focus of each game being a member of x's 1-vn-nhd
    x = np.random.randint(N,size=[2])
    for focus in vn_nhd(x, 1):
        public_goods(vn_nhd(focus,1),t)

    #indirect reciprocity 5x with the recipient of each game being a member of x's 1-vn-nhd
    for focus in vn_nhd(x, 1):
        if spatial == True:
            indirect_reciprocity_spatial(random_neighbour(focus,IR_range), focus, t) 
            #focus = recipient, random player in lattice = donor
        else:
            indirect_reciprocity(random_neighbour(focus,IR_range), focus, t)

    #x decides whether to emulate neighbour's strategy or not
    update_rule(x,random_neighbour(x,1),t)


print("\ntrial", trial, "finished with", no_cooperators[no_rounds],"\n")

print((time.time() - start_time)/3600, "hours")


    


In [ ]:
folder = "PGPixel"

#create line graph to show number of cooperators
x = np.linspace(0,40000,40001)
plt.plot(x, no_cooperators, color="#094A25")
plt.ylabel("Number of Cooperators")
plt.xlabel("Time step (thousands)")
plt.title("Cooperators over Time")
plt.savefig(fname = folder + "/timeseries.jpg")
plt.show()

plt.plot(x,[29 for i in range(40001)], color = "red", linestyle = "--")
plt.plot(x,[8 for i in range(40001)], color = "red", linestyle = "--")
plt.plot(x, no_cooperators, color="#094A25")
plt.ylim(0,20)
plt.xlim(0,40000)
plt.yticks(range(5,35,5))
plt.ylabel("Number of Cooperators")
plt.xlabel("Time step (thousands)")
plt.title("Cooperators over Time")
plt.savefig(fname = folder + "/timeseries_zoom.jpg")
plt.show()


extinct = np.where(no_cooperators==0)[0]
print(extinct[0])

#plot each player as pixel
for plot_t in range(0,40001,5000):
    plt.imshow(strategy[plot_t], cmap = cmap, vmin = 0, vmax = 1)
    plt.title(f"t = {plot_t}")
    plt.xticks([10*i for i in range(round(N/10))])
    plt.yticks([10*i for i in range(round(N/10))])
    #plt.xlim(10,20)
    #plt.ylim(30,40)
    plt.xticks([], [])
    plt.yticks([], [])
    plt.savefig(fname = folder + "/" + str(plot_t) + ".jpg")
    plt.show()